In [1]:
import pickle, networkx as nx, matplotlib.pyplot as plt
import random
import itertools
import networkx as nx
import pprint                     # for nicer printing
import csv
from pathlib import Path

In [2]:
from dgd.environments.drl3env_loader6 import _apply_implicit_or, _compute_hash, _compute_truth_key

from dgd.utils.utils5 import (
    calculate_truth_table_v2,
    generate_one_hot_features_from_adj,
    resize_matrix,
    energy_score,
    check_implicit_OR_existence_v3,
    add_implicit_OR_to_dag_v2,
    exhaustive_cut_enumeration_dag,
    is_fanout_free_standalone,
    generate_subgraph,
    simulate_signal_propagation_binary,
    write_tt_files,
    plot_circuit_layered
)

from dgd.utils.verification import *

Done loading action motifs. There are 15928 unique motifs.


In [3]:
with open('/home/gridsan/spalacios/DRL1/supercloud-testing/ABC-and-PPO-testing1/4_input_precomputed_graphs/graphs_library_4_input_ABC.pkl', 'rb') as file:
    graphs_library_4_input_ABC = pickle.load(file)

In [4]:
OUT_DIR  = Path(
    "/home/gridsan/spalacios/Designing complex biological circuits with deep neural networks/"
    "dgd/data/ABC_designs"
)
OUT_DIR.mkdir(parents=True, exist_ok=True)          # ensure it exists
CSV_PATH = OUT_DIR / "ABC_energies_integrating_OR_search.csv"

In [5]:
input_signals_list_binary = [
    {0: 0, 1: 0, 2: 0, 3: 0},  
    {0: 0, 1: 0, 2: 0, 3: 1},  
    {0: 0, 1: 0, 2: 1, 3: 0}, 
    {0: 0, 1: 0, 2: 1, 3: 1}, 
    {0: 0, 1: 1, 2: 0, 3: 0},  
    {0: 0, 1: 1, 2: 0, 3: 1},    
    {0: 0, 1: 1, 2: 1, 3: 0},  
    {0: 0, 1: 1, 2: 1, 3: 1}, 
    {0: 1, 1: 0, 2: 0, 3: 0},  
    {0: 1, 1: 0, 2: 0, 3: 1},  
    {0: 1, 1: 0, 2: 1, 3: 0}, 
    {0: 1, 1: 0, 2: 1, 3: 1}, 
    {0: 1, 1: 1, 2: 0, 3: 0},  
    {0: 1, 1: 1, 2: 0, 3: 1},    
    {0: 1, 1: 1, 2: 1, 3: 0},  
    {0: 1, 1: 1, 2: 1, 3: 1},     
]    

In [6]:
#This cover random first part
#0x000D - 0x6572

decimal_inputs_1 = [
     13,   569,   772,  1035,  1397,  1402,  1603,  1888,  2479,  3906,
   4152,  4168,  4297,  4740,  4899,  5070,  5908,  6232,  6752,  6854,
   7359,  7573,  8158,  8811,  8902,  9127,  9231, 10808, 10838, 12231,
  12384, 12494, 12970, 13763, 14044, 14354, 14871, 15153, 15200, 15208,
  16539, 16802, 16818, 17051, 18212, 18429, 18625, 18994, 19448, 21013,
  21423, 21463, 22938, 23213, 23344, 23977, 24321, 24546, 24605, 24672,
  24938, 25739, 25970
]

#This cover random second part
#0x680A - 0xFC79
decimal_inputs_2 = [
    26634, 26695, 27037, 28458, 28822, 28908, 29046, 33323,
    34062, 36707, 37196, 37258, 37804, 38289, 38647, 39191,
    39925, 40842, 41690, 42930, 43360, 46916, 47277, 48150,
    48291, 48625, 48873, 48950, 49736, 50354, 51046, 52098,
    52182, 52887, 54041, 54054, 54391, 54500, 54608, 55936,
    56314, 58885, 58999, 59706, 60657, 61419, 62527, 62695,
    62884, 64633]


all_keys = decimal_inputs_1 + decimal_inputs_2     

#all_keys = [3847]

# -------------------------------------------------
# 2)  accumulate rows   (key, energy, …metrics…)
# -------------------------------------------------
rows = []
for k in all_keys:
    g = graphs_library_4_input_ABC.get(k)
    if g is None:
        print(f" graph {k} not found — skipping")
        continue

    # ----- energy ---------------------------------------------------
    energy_val, metrics = energy_score(g, check_implicit_OR_existence_v3)

    # ----- simulation & truth-table ---------------------------------
    sim_raw = simulate_signal_propagation_binary(g, input_signals_list_binary)
    tt_raw  = calculate_truth_table_v2(g)

    sim_out = [list(d.values())[0] for d in sim_raw]      # [{4:0},…] → [0,…]
    tt_out  = [v[0] for v in tt_raw.values()]             # {(0,1,0,1):(1,),…}

    # bit-strings → hex (padded)
    sim_bits = ''.join(map(str, sim_out))
    tt_bits  = ''.join(map(str, tt_out))

    sim_bits = sim_bits.zfill((len(sim_bits)+3)//4*4)
    tt_bits  = tt_bits .zfill((len(tt_bits )+3)//4*4)

    sim_hex = '0x' + format(int(sim_bits, 2),
                            f'0{len(sim_bits)//4}X')
    tt_hex  = '0x' + format(int(tt_bits , 2),
                            f'0{len(tt_bits )//4}X')

    # ----- assemble row --------------------------------------------
    rows.append({
        "key"      : k,
        "hex"      : f"0x{k:04X}",        # requested hex column
        "energy"   : energy_val,
        **metrics,                       # num_nodes, … etc.
        "sim_hex"  : sim_hex,
        "tt_hex"   : tt_hex,
    })
    
    ABC_design_original = g.copy()
    ABC_design = _apply_implicit_or(g, 2)
    
    # Structural verification 
    result = validate_dag(ABC_design, 1, raise_on_error=True, plot_on_error=True)
    print(result)
        
    # Functional Verification 
    eq_folder = OUT_DIR / f"{tt_hex}_yosys_eqv"
    eq_folder.mkdir(parents=True, exist_ok=True)  # ensure path exists

    ok_tt = validate_tt_using_yosys(ABC_design, folder=eq_folder, verbose=True)
    print(ok_tt)
    if not ok_tt:
        raise AssertionError(
            f"Truth-table mismatch. See {eq_folder}/truth.out and tt_*.json/txt."
        )   
    
    #Compute and export truth table    
    tt_folder = OUT_DIR/ f"{tt_hex}_tt"  # fixed quotes
    tt = calculate_truth_table_v2(ABC_design)
    write_tt_files(ABC_design, tt, tt_folder)    
    
    #Compute truth table using Yosys 
    yosys_tt_folder =OUT_DIR/ f"{tt_hex}_yosys_tt" 
    combos, yosys_rows = yosys_truth_table_with_yosys(ABC_design, folder = yosys_tt_folder, verbose=True)
    in_names, out_names = infer_io(ABC_design)  
    write_tt_files_yosys(yosys_tt_folder, combos, yosys_rows, in_names, out_names)
    
    plot_circuit_layered(
        ABC_design,
        outdir=OUT_DIR,
        name=f'{tt_hex}_optimal_topology',
        fmt="pdf",
        rankdir="LR",
        show_in_notebook=False,
        compact=True,         
        gv_size="6,4!",            
        gv_dpi=None,                 
        gv_margin="0.05",
        gv_pad="0.05"
    )              

    plot_circuit_layered(
        ABC_design_original,
        outdir=OUT_DIR,
        name=f'{tt_hex}_optimal_topology_original',
        fmt="pdf",
        rankdir="LR",
        show_in_notebook=False,
        compact=True,         
        gv_size="6,4!",            
        gv_dpi=None,                 
        gv_margin="0.05",
        gv_pad="0.05"
    )               

# consistent column order
fieldnames = ["key", "hex", "energy"] + sorted(
    {fld for r in rows for fld in r if fld not in ("key","hex","energy")}
)

# ------------------------------------------------------------------
# 3)  write CSV
# ------------------------------------------------------------------
with CSV_PATH.open("w", newline="", encoding="utf-8") as fh:
    writer = csv.DictWriter(fh, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(rows)

print(f"Saved {len(rows)} designs to {CSV_PATH.resolve()}")

{'ok': True, 'inputs': [0, 1, 2, 3], 'outputs': [4], 'errors': [], 'warnings': []}
[test] inputs=[0, 1, 2, 3]  outputs=[4]
[test] wrote /home/gridsan/spalacios/Designing complex biological circuits with deep neural networks/dgd/data/ABC_designs/0x000D_yosys_eqv/gate.v
Truth tables match for all 16 input combinations.
True
[Yosys] using: /home/gridsan/spalacios/.local/bin/yowasp-yosys
0 | 1 | 2 | 3 | 4
-----------------
0 0 0 0 | 0
0 0 0 1 | 0
0 0 1 0 | 0
0 0 1 1 | 0
0 1 0 0 | 0
0 1 0 1 | 0
0 1 1 0 | 0
0 1 1 1 | 0
1 0 0 0 | 0
1 0 0 1 | 0
1 0 1 0 | 0
1 0 1 1 | 0
1 1 0 0 | 1
1 1 0 1 | 1
1 1 1 0 | 0
1 1 1 1 | 1
{'ok': True, 'inputs': [0, 1, 2, 3], 'outputs': [4], 'errors': [], 'warnings': []}
[test] inputs=[0, 1, 2, 3]  outputs=[4]
[test] wrote /home/gridsan/spalacios/Designing complex biological circuits with deep neural networks/dgd/data/ABC_designs/0x0239_yosys_eqv/gate.v
Truth tables match for all 16 input combinations.
True
[Yosys] using: /home/gridsan/spalacios/.local/bin/yowasp-yosy